In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/detection/images/train/623.jpg
/kaggle/input/detection/images/train/208.jpg
/kaggle/input/detection/images/train/473.jpg
/kaggle/input/detection/images/train/128.xml
/kaggle/input/detection/images/train/200.xml
/kaggle/input/detection/images/train/333.jpg
/kaggle/input/detection/images/train/537.jpg
/kaggle/input/detection/images/train/407.xml
/kaggle/input/detection/images/train/45.jpg
/kaggle/input/detection/images/train/369.jpg
/kaggle/input/detection/images/train/56.jpg
/kaggle/input/detection/images/train/557.xml
/kaggle/input/detection/images/train/654.jpg
/kaggle/input/detection/images/train/346.xml
/kaggle/input/detection/images/train/89.jpg
/kaggle/input/detection/images/train/20.jpg
/kaggle/input/detection/images/train/275.jpg
/kaggle/input/detection/images/train/560.xml
/kaggle/input/detection/images/train/212.jpg
/kaggle/input/detection/images/train/701.xml
/kaggle/input/detection/images/train/239.jpg
/kaggle/input/detection/images/train/655.xml
/kaggle/input/

In [22]:
import os
import cv2
import tensorflow as tf
import pytesseract as pt
import plotly.express as px
import matplotlib.pyplot as plt
import xml.etree.ElementTree as xet
from glob import glob
from skimage import io
from shutil import copy
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2 
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [3]:
path = glob('/kaggle/input/detection/images/train/*.xml')
labels_dict = dict(filepath=[],xmin=[],xmax=[],ymin=[],ymax=[])
for filename in path:
    info = xet.parse(filename)
    root = info.getroot()
    member_object = root.find('object')
    labels_info = member_object.find('bndbox')
    xmin = int(labels_info.find('xmin').text)
    xmax = int(labels_info.find('xmax').text)
    ymin = int(labels_info.find('ymin').text)
    ymax = int(labels_info.find('ymax').text)

    labels_dict['filepath'].append(filename)
    labels_dict['xmin'].append(xmin)
    labels_dict['xmax'].append(xmax)
    labels_dict['ymin'].append(ymin)
    labels_dict['ymax'].append(ymax)

In [4]:
df = pd.DataFrame(labels_dict)
df.head()

,filepath,xmin,xmax,ymin,ymax
0,/kaggle/input/detection/images/train/128.xml,403,585,432,474
1,/kaggle/input/detection/images/train/200.xml,808,1092,857,930
2,/kaggle/input/detection/images/train/407.xml,69,214,481,550
3,/kaggle/input/detection/images/train/557.xml,436,648,490,548
4,/kaggle/input/detection/images/train/346.xml,453,610,841,878


In [7]:

def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('/kaggle/input/detection/images/train',filename_image)
    return filepath_image
getFilename(filename)

'/kaggle/input/detection/images/train/393.jpg'

In [8]:
image_path = list(df['filepath'].apply(getFilename))

In [9]:
img_arr = cv2.imread(image_path[0])

In [10]:
labels = df.iloc[:,1:].values
data = []
output = []
for ind in range(len(image_path)):
    image = image_path[ind]
    img_arr = cv2.imread(image)
    img_arr=np.array(img_arr)
    h,w,d = img_arr.shape
    # Prepprocesing
    load_image = load_img(image,target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr/255.0 # Normalization
    # Normalization to labels
    xmin,xmax,ymin,ymax = labels[ind]
    nxmin,nxmax = xmin/w,xmax/w
    nymin,nymax = ymin/h,ymax/h
    label_norm = (nxmin,nxmax,nymin,nymax) # Normalized output
    # Append
    data.append(norm_load_image_arr)
    output.append(label_norm)

In [11]:
X = np.array(data,dtype=np.float32)
y = np.array(output,dtype=np.float32)

In [12]:
x_train,x_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=0)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((567, 224, 224, 3), (142, 224, 224, 3), (567, 4), (142, 4))

In [26]:
model=MobileNetV2(include_top=False,input_shape = (224,224,3),weights='imagenet')
flatten = Flatten()(model.output)
bbox = Dense(4)(flatten)
final=Model(inputs = model.input , outputs = bbox)
final.compile(loss='mse', optimizer='adam')
final.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_5[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                           

In [28]:
done=final.fit(x_train,y_train,batch_size=32,epochs=60,validation_data = (x_test , y_test))

Epoch 1/60
18/18 [==============================] - 2s 106ms/step - loss: 0.0136 - val_loss: 5.9149
Epoch 2/60
18/18 [==============================] - 2s 94ms/step - loss: 0.0090 - val_loss: 5.6024
Epoch 3/60
18/18 [==============================] - 2s 96ms/step - loss: 0.0076 - val_loss: 4.8313
Epoch 4/60
18/18 [==============================] - 2s 94ms/step - loss: 0.0079 - val_loss: 4.3270
Epoch 5/60
18/18 [==============================] - 2s 95ms/step - loss: 0.0098 - val_loss: 4.0515
Epoch 6/60
18/18 [==============================] - 2s 95ms/step - loss: 0.0112 - val_loss: 3.2778
Epoch 7/60
18/18 [==============================] - 2s 94ms/step - loss: 0.0095 - val_loss: 2.6936
Epoch 8/60
18/18 [==============================] - 2s 94ms/step - loss: 0.0099 - val_loss: 2.2210
Epoch 9/60
18/18 [==============================] - 2s 95ms/step - loss: 0.0075 - val_loss: 2.0204
Epoch 10/60
18/18 [==============================] - 2s 95ms/step - loss: 0.0075 - val_loss: 1.8284
Epoch 11

In [39]:
from PIL import Image
path_testing = glob('/kaggle/input/testing/Cars/*.jpg')
testing_dict = dict(filepath=[],xmin=[],xmax=[],ymin=[],ymax=[])
for filename in path_testing:
    img_arr = Image.open(filename)
    w,h=img_arr.size
    re_img=img.resize((224,224))
    img=np.array(re_img)
    img= img / 255.0
    pred=final.predict(img)
    x,y,z,a=int(pred[0])*w,int(pred[1])*y,int(pred[2])*x,int(pred[3])*y
    cv2.rectangle(img , (x,y) , (z,a) , (255,0,0) , 2)
    
    # Display the Image
    display(img)
    display(img.fromarray(img))

TypeError: unsupported operand type(s) for /: 'NoneType' and 'float'